In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
from __future__ import print_function, division
import copy
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torchvision.transforms import ToTensor,Resize,Normalize,RandomHorizontalFlip,RandomVerticalFlip,CenterCrop
from sklearn.metrics import accuracy_score,precision_recall_fscore_support,roc_curve,auc,roc_auc_score,classification_report
import matplotlib.pyplot as plt
import time
import os
import copy

cudnn.benchmark = True
plt.ion()   # interactive mode

In [3]:
print(torch.cuda.device_count())
print(torch.cuda.current_device())

1
0


Run for the first iteration to generate base Global Model

In [4]:

method = 'FedAdaGrad'
i = 10
learningRate = 0.001
c1  = 1 
c2 = 1
c3 = 1
c4 = 1
#dd '''2525 246 1439'''
base_weight_dir = '/DATA1/Mangaldeep/V3/Weight/FedAdaGrad'
base_log_dir = '/DATA1/Mangaldeep/V3/Log/FedAdaGrad'
avg_weight_dir = base_weight_dir+'/'+'avg'+str(i)+'.pth'
logfilepath = base_log_dir+'/'+str(method)+'_'+str(i)+'.txt'
if i >0 :
    previous_avg_dir = base_weight_dir+'/'+'avg'+str(i-1)+'.pth'

In [5]:
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

if i == 0:
    model_ft = models.efficientnet_b3(weights=None)
    num_ftrs = model_ft.classifier[1].in_features
    model_ft.classifier[1] = nn.Linear(num_ftrs, 2)
    torch.save(model_ft.state_dict(),avg_weight_dir)

cuda:0


In [6]:
data1_dir = '/DATA1/Mangaldeep/V3/Dataset/BRACS1/test'
data2_dir = '/DATA1/Mangaldeep/V3/Dataset/BRACS2/test'
data3_dir = '/DATA1/Mangaldeep/V3/Dataset/BreakHis1/test'
data4_dir = '/DATA1/Mangaldeep/V3/Dataset/BreakHis2/test'

In [7]:
weight1_dir = '/DATA1/Mangaldeep/V3/Weight/FedAdaGrad/FedAdaGrad_BRACS1_'+str(i-1)+'_1323.pth'
weight2_dir = '/DATA1/Mangaldeep/V3/Weight/FedAdaGrad/FedAdaGrad_BRACS2_'+str(i-1)+'_1323.pth'
weight3_dir = '/DATA1/Mangaldeep/V3/Weight/FedAdaGrad/FedAdaGrad_BreakHis1_'+str(i-1)+'_697.pth'
weight4_dir = '/DATA1/Mangaldeep/V3/Weight/FedAdaGrad/FedAdaGrad_BreakHis2_'+str(i-1)+'_698.pth'

In [8]:
weight1 = torch.load(weight1_dir,map_location=torch.device('cpu'))
weight2 = torch.load(weight2_dir,map_location=torch.device('cpu'))
weight3 = torch.load(weight3_dir,map_location=torch.device('cpu'))
weight4 = torch.load(weight4_dir,map_location=torch.device('cpu'))
weight_avg = torch.load(weight3_dir,map_location=torch.device('cpu'))


total = c1 + c2+ c3 +c4
for key in weight1:
  weight_avg[key] = (weight1[key] + weight2[key] + weight3[key] + weight4[key] )/total

In [9]:
##Global Model https://github.com/lxcnju/FedRepo/blob/main/algorithms/fedopt.py

global_model = models.efficientnet_b3(weights=None)
num_ftrs = global_model.classifier[1].in_features
global_model.classifier[1] = nn.Linear(num_ftrs, 2)
global_model.load_state_dict(torch.load(previous_avg_dir))
global_model = global_model.to(device)

## Global Optimizer

global_optimizer = optim.Adagrad(global_model.parameters(), lr=learningRate) #,weight_decay=0.005
#previous weight decay 0.00001 lr=0.0002 till avg4

In [10]:
# zero_grad
global_optimizer.zero_grad()
global_optimizer_state = global_optimizer.state_dict()

# new_model
new_model = copy.deepcopy(global_model)
new_model.load_state_dict(weight_avg, strict=True)

# set global_model gradient
with torch.no_grad():
    for param, new_param in zip(
        global_model.parameters(), new_model.parameters()
        ):
        param.grad = param.data - new_param.data

# replace some non-parameters's state dict
state_dict = global_model.state_dict()
for name in dict(global_model.named_parameters()).keys():
    weight_avg[name] = state_dict[name]
global_model.load_state_dict(weight_avg, strict=True)

# optimization
global_optimizer.load_state_dict(global_optimizer_state)
global_optimizer.step()

In [11]:
torch.save(global_model.state_dict(),avg_weight_dir)
data_transforms = {
    'train': transforms.Compose([
        Resize((256,256),interpolation=transforms.InterpolationMode.BICUBIC),
        transforms.RandomCrop(224),
        #transforms.RandomResizedCrop(size = (224,224)),
        #transforms.CenterCrop(224),
        transforms.RandomHorizontalFlip(p=0.3),
        transforms.RandomVerticalFlip(p=0.3),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        #transforms.Resize(256),
        Resize((256,256),interpolation=transforms.InterpolationMode.BICUBIC),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

model_ft = models.efficientnet_b3(weights=None)
num_ftrs = model_ft.classifier[1].in_features
model_ft.classifier[1] = nn.Linear(num_ftrs, 2)

model_ft.load_state_dict(torch.load(avg_weight_dir))
#model_ft = model_ft.to(device)


model_ft = model_ft.to(device)
model_ft.eval()
criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
#optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.0001, momentum=0.9)
optimizer_ft = optim.Adam(model_ft.parameters(), lr=0.001)
# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.5)

# BreakHis1

In [12]:
test_dataset = datasets.ImageFolder(data3_dir, transform=data_transforms['val'])
class_names = test_dataset.classes
test_loader = torch.utils.data.DataLoader(
        test_dataset,
        batch_size=1,
        num_workers=0,
        shuffle=False
    )
model_ft.eval()
with torch.no_grad():
  y_true=[]
  y_pred=[]
  y_probas = []
  for i, (inputs, labels) in enumerate(test_loader):
    inputs = inputs.to(device)
    labels = labels.to(device)
    y_true.append(labels.detach().cpu().numpy())
    model_ft.to(device)
    outputs = model_ft(inputs)
    outputs = nn.Softmax(dim=1)(outputs)
    max, preds = torch.max(outputs, 1)
    y_probas.append(outputs.cpu().numpy())
    y_pred.append(int(preds.detach().cpu().numpy()))


y_true = np.array(y_true)
y_pred = np.array(y_pred)
prob = np.array(y_probas)
acc = accuracy_score(y_true,y_pred)
y_probas = prob.reshape(prob.shape[0],prob.shape[2])
precision,recall,fscore,_=precision_recall_fscore_support(y_true, y_pred, average='macro')
auc_score = roc_auc_score(y_true, y_probas[:,1])
precision,recall,fscore,_=precision_recall_fscore_support(y_true, y_pred, average='macro')
print(f"BreakHis1 accuracy = {round(acc,4)} , precision = {round(precision,4)},recall = {round(recall,4)},fscore = {round(fscore,4)},,auc_score = {round(auc_score,4)}")
print(classification_report(y_true,y_pred))
with open (logfilepath, 'a') as file:
    file.writelines(f"Learning rate {learningRate}")
    file.writelines('\n')
    file.writelines(data3_dir)
    file.writelines('\n')
    file.writelines(f"BreakHis1 accuracy = {round(acc,4)} , precision = {round(precision,4)},recall = {round(recall,4)},fscore = {round(fscore,4)},auc_score = {round(auc_score,4)}")
    file.writelines('\n')
    file.writelines(classification_report(y_true,y_pred))
    file.writelines('\n')

BreakHis1 accuracy = 0.685 , precision = 0.3425,recall = 0.5,fscore = 0.4065,,auc_score = 0.5513
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        63
           1       0.69      1.00      0.81       137

    accuracy                           0.69       200
   macro avg       0.34      0.50      0.41       200
weighted avg       0.47      0.69      0.56       200



/home/banerjee1/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/banerjee1/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/banerjee1/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/banerjee1/.local/lib/pyth

# BreakHis2

In [13]:
test_dataset = datasets.ImageFolder(data4_dir, transform=data_transforms['val'])
class_names = test_dataset.classes
test_loader = torch.utils.data.DataLoader(
        test_dataset,
        batch_size=1,
        num_workers=0,
        shuffle=False
    )
model_ft.eval()
with torch.no_grad():
  y_true=[]
  y_pred=[]
  y_probas = []
  for i, (inputs, labels) in enumerate(test_loader):
    inputs = inputs.to(device)
    labels = labels.to(device)
    y_true.append(labels.detach().cpu().numpy())
    model_ft.to(device)
    outputs = model_ft(inputs)
    outputs = nn.Softmax(dim=1)(outputs)
    max, preds = torch.max(outputs, 1)
    y_probas.append(outputs.cpu().numpy())
    y_pred.append(int(preds.detach().cpu().numpy()))


y_true = np.array(y_true)
y_pred = np.array(y_pred)
prob = np.array(y_probas)
acc = accuracy_score(y_true,y_pred)
y_probas = prob.reshape(prob.shape[0],prob.shape[2])
precision,recall,fscore,_=precision_recall_fscore_support(y_true, y_pred, average='macro')
auc_score = roc_auc_score(y_true, y_probas[:,1])
precision,recall,fscore,_=precision_recall_fscore_support(y_true, y_pred, average='macro')
print(f"BreakHis2 accuracy = {round(acc,4)} , precision = {round(precision,4)},recall = {round(recall,4)},fscore = {round(fscore,4)},,auc_score = {round(auc_score,4)}")
print(classification_report(y_true,y_pred))
with open (logfilepath, 'a') as file:
    file.writelines(f"Learning rate {learningRate}")
    file.writelines('\n')
    file.writelines(data3_dir)
    file.writelines('\n')
    file.writelines(f"BreakHis2 accuracy = {round(acc,4)} , precision = {round(precision,4)},recall = {round(recall,4)},fscore = {round(fscore,4)},auc_score = {round(auc_score,4)}")
    file.writelines('\n')
    file.writelines(classification_report(y_true,y_pred))
    file.writelines('\n')

BreakHis2 accuracy = 0.6866 , precision = 0.3433,recall = 0.5,fscore = 0.4071,,auc_score = 0.5705
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        63
           1       0.69      1.00      0.81       138

    accuracy                           0.69       201
   macro avg       0.34      0.50      0.41       201
weighted avg       0.47      0.69      0.56       201



/home/banerjee1/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/banerjee1/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/banerjee1/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/banerjee1/.local/lib/pyth

## BRACS1

In [14]:
test_dataset = datasets.ImageFolder(data1_dir, transform=data_transforms['val'])
class_names = test_dataset.classes
test_loader = torch.utils.data.DataLoader(
        test_dataset,
        batch_size=1,
        num_workers=0,
        shuffle=False
    )
model_ft.eval()
with torch.no_grad():
  y_true=[]
  y_pred=[]
  y_probas = []
  for i, (inputs, labels) in enumerate(test_loader):
    inputs = inputs.to(device)
    labels = labels.to(device)
    y_true.append(labels.detach().cpu().numpy())
    model_ft.to(device)
    outputs = model_ft(inputs)
    outputs = nn.Softmax(dim=1)(outputs)
    max, preds = torch.max(outputs, 1)
    y_probas.append(outputs.cpu().numpy())
    y_pred.append(int(preds.detach().cpu().numpy()))


y_true = np.array(y_true)
y_pred = np.array(y_pred)
prob = np.array(y_probas)
acc = accuracy_score(y_true,y_pred)
y_probas = prob.reshape(prob.shape[0],prob.shape[2])
precision,recall,fscore,_=precision_recall_fscore_support(y_true, y_pred, average='macro')
auc_score = roc_auc_score(y_true, y_probas[:,1])
precision,recall,fscore,_=precision_recall_fscore_support(y_true, y_pred, average='macro')
print(f"BRACS1 accuracy = {round(acc,4)} , precision = {round(precision,4)},recall = {round(recall,4)},fscore = {round(fscore,4)},,auc_score = {round(auc_score,4)}")
print(classification_report(y_true,y_pred))
with open (logfilepath, 'a') as file:
    file.writelines(data1_dir)
    file.writelines('\n')
    file.writelines(f"BRACS1 accuracy = {round(acc,4)} , precision = {round(precision,4)},recall = {round(recall,4)},fscore = {round(fscore,4)},,auc_score = {round(auc_score,4)}")
    file.writelines('\n')
    file.writelines(classification_report(y_true,y_pred))
    file.writelines('\n')


BRACS1 accuracy = 0.4069 , precision = 0.2034,recall = 0.5,fscore = 0.2892,,auc_score = 0.655
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       121
           1       0.41      1.00      0.58        83

    accuracy                           0.41       204
   macro avg       0.20      0.50      0.29       204
weighted avg       0.17      0.41      0.24       204



/home/banerjee1/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/banerjee1/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/banerjee1/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/banerjee1/.local/lib/pyth

# BRACS2

In [15]:
test_dataset = datasets.ImageFolder(data2_dir, transform=data_transforms['val'])
class_names = test_dataset.classes
test_loader = torch.utils.data.DataLoader(
        test_dataset,
        batch_size=1,
        num_workers=0,
        shuffle=False
    )
model_ft.eval()
with torch.no_grad():
  y_true=[]
  y_pred=[]
  y_probas = []
  for i, (inputs, labels) in enumerate(test_loader):
    inputs = inputs.to(device)
    labels = labels.to(device)
    y_true.append(labels.detach().cpu().numpy())
    model_ft.to(device)
    outputs = model_ft(inputs)
    outputs = nn.Softmax(dim=1)(outputs)
    max, preds = torch.max(outputs, 1)
    y_probas.append(outputs.cpu().numpy())
    y_pred.append(int(preds.detach().cpu().numpy()))


y_true = np.array(y_true)
y_pred = np.array(y_pred)
prob = np.array(y_probas)
acc = accuracy_score(y_true,y_pred)
y_probas = prob.reshape(prob.shape[0],prob.shape[2])
precision,recall,fscore,_=precision_recall_fscore_support(y_true, y_pred, average='macro')
auc_score = roc_auc_score(y_true, y_probas[:,1])
precision,recall,fscore,_=precision_recall_fscore_support(y_true, y_pred, average='macro')
print(f"BRACS2 accuracy = {round(acc,4)} , precision = {round(precision,4)},recall = {round(recall,4)},fscore = {round(fscore,4)},,auc_score = {round(auc_score,4)}")
print(classification_report(y_true,y_pred))
with open (logfilepath, 'a') as file:
    file.writelines(data1_dir)
    file.writelines('\n')
    file.writelines(f"BRACS2 accuracy = {round(acc,4)} , precision = {round(precision,4)},recall = {round(recall,4)},fscore = {round(fscore,4)},,auc_score = {round(auc_score,4)}")
    file.writelines('\n')
    file.writelines(classification_report(y_true,y_pred))
    file.writelines('\n')


BRACS2 accuracy = 0.4069 , precision = 0.2034,recall = 0.5,fscore = 0.2892,,auc_score = 0.6803
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       121
           1       0.41      1.00      0.58        83

    accuracy                           0.41       204
   macro avg       0.20      0.50      0.29       204
weighted avg       0.17      0.41      0.24       204



/home/banerjee1/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/banerjee1/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/banerjee1/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/banerjee1/.local/lib/pyth